In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import nltk
import re

stop_words = nltk.corpus.stopwords.words('english')

In [2]:
df = pd.read_csv(r'.\booklist.csv', index_col=False)
print(df['title'].head(10))
print(df.shape)
df = df.sample(n=20000)
print(df['title'].head(10))
print(df.shape)

col_names=['bookId','title','series','author','rating','description','lang','isbn','genres','characters','bookForm','edition','pages','publisher','publishDate','firstPublished','awards','numRating','ratingsByStars','likedPercent','setting','coverImg','bbeScore','bbeVotes','price']
wanted_cols = ['bookId','title','series','author','rating','description','genres','characters','setting','coverImg']

#TODO: filter out our wanted cols, create our index and save it to a file(?)
filtered_df = df[wanted_cols].head(5)

print(df['genres'].dtypes)
print(df['characters'].dtypes)
print(df['setting'].dtypes)

0                                     The Hunger Games
1            Harry Potter and the Order of the Phoenix
2                                To Kill a Mockingbird
3                                  Pride and Prejudice
4                                             Twilight
5                                       The Book Thief
6                                          Animal Farm
7                             The Chronicles of Narnia
8    J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
9                                   Gone with the Wind
Name: title, dtype: object
(52478, 25)
17163                                 Dirty English
23564                                    The Escape
20904             Visual Perception with Tensorflow
21610                            Romiette and Julio
18913    The Divine Family: Experiential Narratives
8326                                 Galaxy Pirates
12378                             Swallowing Stones
17077                                On Mystic 

In [3]:
# Characters, author and genre
df['genres'] = df['genres'].apply(literal_eval)
df['characters'] = df['characters'].apply(literal_eval)
df['setting'] = df['setting'].apply(literal_eval)
df['author'] = df['author'].apply(lambda x : x.split(", "))
print(df['author'].dtypes)
print(df['description'].dtypes)

df['soup'] = df['author'] + df['genres'] + df['characters'] + df['setting'] 
df['soup'] = df['soup'].apply(lambda x: ' '.join(x))


object
object


In [4]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', ' ', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [5]:
df['description'] = df['description'].apply(lambda x : normalize_document(str(x)))


df['soup'] = df['soup'] + " " + df['description']
print(df['description'].iloc[0])
print(df['soup'].iloc[0])

scarred fighter girl rules one night unbridled passion three things need know elizabeth bennett smart whip always control lives set carefully crafted rules learned hard way people love always hurt end meets declan blay new neighbor apartment complex tattooed british street fighter campus bad boy supposed avoid saves frat party gone bad rules sex love fly window gives one night unbridled passion longs cardboard thin wall separating bedrooms dreams possessing vulnerable girl next door forever one night two damaged hearts passion lifetime modern love story inspired pride prejudice
Ilsa Madden-Mills (Goodreads Author) Romance New Adult College Contemporary Contemporary Romance Fighters Sports Abuse Sports Romance Adult scarred fighter girl rules one night unbridled passion three things need know elizabeth bennett smart whip always control lives set carefully crafted rules learned hard way people love always hurt end meets declan blay new neighbor apartment complex tattooed british street f

In [6]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['soup'].values.astype('U'))

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['soup'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [7]:
print(cosine_sim)
print(cosine_sim.shape)
indices = pd.Series([i for i in range(len(df))] ,index=df['title'])
print(indices.head(10))
print(indices.shape)
print(cosine_sim)

[[1.00000000e+00 1.10858255e-02 3.87142290e-03 ... 1.68103888e-02
  2.78246393e-02 1.14507389e-02]
 [1.10858255e-02 1.00000000e+00 2.87558966e-03 ... 3.04158039e-03
  2.82451787e-03 8.06185697e-04]
 [3.87142290e-03 2.87558966e-03 1.00000000e+00 ... 0.00000000e+00
  4.69141312e-03 5.51289275e-03]
 ...
 [1.68103888e-02 3.04158039e-03 0.00000000e+00 ... 1.00000000e+00
  1.83381277e-02 9.57419676e-03]
 [2.78246393e-02 2.82451787e-03 4.69141312e-03 ... 1.83381277e-02
  1.00000000e+00 5.47205866e-03]
 [1.14507389e-02 8.06185697e-04 5.51289275e-03 ... 9.57419676e-03
  5.47205866e-03 1.00000000e+00]]
(20000, 20000)
title
Dirty English                                 0
The Escape                                    1
Visual Perception with Tensorflow             2
Romiette and Julio                            3
The Divine Family: Experiential Narratives    4
Galaxy Pirates                                5
Swallowing Stones                             6
On Mystic Lake                             

In [8]:
def get_recommendations(title, no_of_recommendation):
    idx = indices[title]
    df['similarity score'] = np.array(cosine_sim[idx])
    df['weightedRating'] = (((df["rating"] -  1)/4) + (df["likedPercent"] * 0.01) + ((df['numRatings'] - df['numRatings'].min()) / (df['numRatings'].max() - df['numRatings'].min())))/3
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:no_of_recommendation+1]
    book_indices = [i[0] for i in sim_scores]
    book_cossim = df[["title", "similarity score", "weightedRating"]].iloc[book_indices]
    book_cossim = book_cossim.sort_values("weightedRating", ascending=False)
    return book_cossim

In [9]:
print(df['title'].iloc[0])
get_recommendations(df['title'].iloc[0], 10)

Dirty English


,title,similarity score,weightedRating
46343,The Connection,0.054956,0.600926
9741,Too Consumed,0.054684,0.592836
3423,Mine,0.053351,0.592157
47255,Sacrifice,0.072951,0.590361
45442,Seductive Nights Trilogy Bundle (Seductive Nig...,0.057863,0.585887
41270,Bad Judgment,0.058771,0.564635
42006,The Bad Boy in the Glasses,0.053061,0.562513
20570,Sweetness,0.054299,0.547539
48836,Since Drew,0.053994,0.545907
28361,Wildcard: Volume One,0.054957,0.532547
